In [1]:
import requests
from bs4 import BeautifulSoup as bs
import unicodedata
import pandas as pd
import re
import os
from tqdm import tqdm

In [2]:
url = 'https://forum.portfolio.hu/topics/otp-reszvenyesek-ide/5224?oldal=1&limit=100'

result = requests.get(url)

soup = bs(result.content, 'lxml', from_encoding = 'utf-8')

content = soup.find_all('div', {'class': 'comment'})

In [25]:
def get_topic_stat(soup):
    topic_data = soup.find('div', {'class': 'topic-head'})
    topic_details = topic_data.find('div', {'class': 'pull-left'})
    topic_founder = unicodedata.normalize('NFKD', topic_details.find('a').text)
    topic_start = unicodedata.normalize('NFKD', topic_details.text)
    topic_start = topic_start.replace(topic_founder, '').replace('Topiknyitó:', '')
    topic_title = unicodedata.normalize('NFKD', topic_data.find('div', {'class': 'comment'}).find('h1').text)
    return topic_founder, topic_start, topic_title

def get_first_page(soup):
    links = soup.find('div', {'class': 'topic-navigation'}).find_all('a', {'class': 'page-link'})
    first = [link for link in links if link.text == 'első'][0]
    first = re.sub('.*oldal=(\d+).*', '\\1', first['href'])
    return int(first)

In [28]:
first_page = get_first_page(soup)
topic_stat = get_topic_stat(soup)

In [90]:
get_text = lambda x: unicodedata.normalize('NFKD', x.find('div', {'class': 'text'}).text)
get_username = lambda x: unicodedata.normalize('NFKD', x.find('a', {'class': 'username'}).text)
get_date = lambda x: unicodedata.normalize('NFKD', x.find('span', {'class': 'date'}).text)
get_id = lambda x: unicodedata.normalize('NFKD', x.find('div', {'class': 'id'}).text)
get_prev_id = lambda x: unicodedata.normalize('NFKD', x.find('div', {'class': 'prevcomment'}).find('a').text)
get_prev_name = lambda x: unicodedata.normalize('NFKD', x.find('div', {'class': 'prevcomment'}).text.split(get_prev_id(x))[-1])
get_like = lambda x: unicodedata.normalize('NFKD', x.find('span', {'class': 'vote', 'data-vote': '1'}).text)
get_unlike = lambda x: unicodedata.normalize('NFKD', x.find('span', {'class': 'vote', 'data-vote': '-1'}).text)

In [97]:
def get_user_ranks(comment):
    avatar = comment.find('div', {'class': 'avatar'})['data-content']
    ranks = bs(avatar, 'lxml').find_all('span')
    rank_data = {rank.find('i')['class'][0].replace('icon-', ''): int(rank.text) for rank in ranks if rank.attrs}
    return rank_data

In [141]:
def get_comment_data(comment):
    try:
        text = get_text(comment)
    except:
        text = None
    
    try:
        username = get_username(comment)
    except:
        username = None
    
    try:
        date = get_date(comment)
    except:
        date = None
    
    try:
        comment_id = get_id(comment)
    except:
        comment_id = None
    
    try:
        like = int(get_like(comment))
    except:
        like = None
    
    try:
        unlike = int(get_unlike(comment))
    except:
        unlike = None
    
    try:
        prev_id = get_prev_id(comment)
    except:
        prev_id = None
    
    try:
        prev_name = get_prev_name(comment)
    except:
        prev_name = None
    
    data = {
        'text': text,
        'username': username,
        'date': date,
        'id': comment_id,
        'like': like,
        'unlike': unlike,
        'prev_id': prev_id,
        'prev_name': prev_name
    }
        
    try:
        user_data = get_user_ranks(comment)
    except:
        user_data = {}
        
    data.update(user_data)
    return data

In [98]:
def get_page_data(url):
    result = requests.get(url)

    if result.status_code != 200:
        return False

    soup = bs(result.content, 'lxml', from_encoding = 'utf-8')

    content = soup.find_all('div', {'class': 'comment'})
    
    data = []
    for i, comment in enumerate(content[1:]):
        data.append(get_comment_data(comment))

    return pd.DataFrame(data)

get_page_data('https://forum.portfolio.hu/topics/otp-reszvenyesek-ide/5224?oldal=2&limit=100')

,text,username,date,id,like,unlike,prev_id,prev_name,expert,activity,popularity
0,"\nVera! \nNyugi, várjuk ki a nap végét.\n",gentlemanus,2006. 07. 27. 10:13,#200,0,0,#198,\n ...,4.0,2.0,4.0
1,"\nSzemi, megtisztelnél a figyelmeddel, Ricsit...",sailor,2006. 07. 27. 10:10,#199,0,0,#198,\n ...,4.0,3.0,4.0
2,\n...jelenleg nem úgy tűnik...\n,Vera76,2006. 07. 27. 10:05,#198,0,0,#197,\n ...,3.0,4.0,1.0
3,\nremélem ez a nap már pozitiv lesz!\n,koi,2006. 07. 27. 09:31,#197,0,0,None,None,1.0,1.0,1.0
4,\nJó reggelt! Vajon ma merre indul az oti? A ...,Vera76,2006. 07. 27. 09:09,#196,0,0,None,None,3.0,4.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...
95,"\nLényegében én is erre gondoltam gentle, c...",egyetleneim,2006. 07. 25. 12:43,#105,0,0,#101,\n ...,5.0,3.0,4.0
96,\nvaloszinü szeptember ????mert attol kezdöd...,koi,2006. 07. 25. 12:01,#104,0,0,#102,\n ...,1.0,1.0,1.0
97,\nszerintetek az emberek hány százaléka fog...,koi,2006. 07. 25. 11:59,#103,0,0,#101,\n ...,1.0,1.0,1.0
98,"\nSztem azert, mert altalaban a nyar pangosabb...",akpo,2006. 07. 25. 11:57,#102,0,0,#98,\n ...,5.0,5.0,4.0


In [99]:
fill_url = lambda x: f'https://forum.portfolio.hu/topics/otp-reszvenyesek-ide/5224?oldal={x}&limit=100'

In [132]:
def get_all_msg(pages):
    bad = []

    if not os.path.exists('Data'):
        os.makedirs('Data')

    for page in tqdm(pages):

        data = pd.DataFrame()

        data = get_page_data(fill_url(page))
        if not isinstance(data, pd.DataFrame):
            bad.append(page)

        data.to_csv(os.path.join('Data', f'data_{page}.csv'), sep = ';', encoding = 'utf-8', index = False)

    pd.DataFrame(bad, columns = ['bad_page']).to_csv(os.path.join('Data', f'bad_page.csv'), sep = ';', encoding = 'utf-8', index = False)

In [ ]:
#range(1, 11)
get_all_msg(range(1, first_page))

In [146]:
pd.read_csv(os.path.join('Data', 'bad_page.csv'), sep = ';', encoding = 'utf-8', header = 0)

,bad_page


In [164]:
data = pd.DataFrame()

for page in tqdm(range(1, first_page)):
    data = pd.concat([data, pd.read_csv(os.path.join('Data', f'data_{page}.csv'), sep = ';', encoding = 'utf-8', header = 0)])

data.to_csv(os.path.join('Data', 'data.csv'), sep = ';', encoding = 'utf-8', index = False)

100%|██████████| 11283/11283 [1:30:28<00:00,  2.08it/s]
